# Python OpenCV benchmark
Since the Julia Benchmark package [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl) automatically selects the number of samples, the Python benchmarks (computed using [timeit](https://github.com/python/cpython/blob/3.8/Lib/timeit.py)) are configured according to the number of samples BenchmarkTools takes.

Our tests consist of the following benchmarks:
1. Loading images
2. Saving images
3. Resizing (Scaling) images
4. Rotating Images
5. Getting the histogram (Grayscale Histogram)
6. Equalizing the Grayscale Histogram
7. Converting image to grayscale
8. Changing colorspace to HSV
9. Applying Gaussian Blur

In [ ]:
import numpy as np
import cv2
import timeit

## 1. Loading images

In [3]:
s = """\
img = cv2.imread('images/lighthouse.png', 1)
"""

timeit.timeit(stmt = s, setup = "import cv2", number = 44)

0.9200869000000012

## 2. Saving images

In [4]:
s = """\
cv2.imwrite('imagecopy.png', img)
"""

setup = """\
import cv2
img = cv2.imread('images/lighthouse.png', 1)
"""

timeit.timeit(stmt = s, setup = setup, number = 50)

2.096548299999995

## 3. Resizing (Scaling) images

In [5]:
s = """\
cv2.resize(img, None, fx=2, fy=2, interpolation = cv2.INTER_CUBIC)
"""

setup = """\
import cv2
img = cv2.imread('images/lighthouse.png', 1)
"""

timeit.timeit(stmt = s, setup = setup, number = 22)

0.16705960000001596

## 4. Rotating Images

In [6]:
s = """\
size = img.shape
M = cv2.getRotationMatrix2D((size[0]/2,size[1]/2),90, 1)
dst = cv2.warpAffine(img,M,(size[0],size[1]))
"""

setup = """\
import cv2
img = cv2.imread('images/lighthouse.png', 1)
"""

timeit.timeit(stmt = s, setup = setup, number = 184)

0.7835640000000694

## 5. Getting the histogram (Grayscale Histogram)

In [7]:
s = """\
hist = cv2.calcHist([img],[0],None,[256],[0,256])
"""

setup = """\
import cv2
img = cv2.imread('images/lighthouse.png', 1)
"""

timeit.timeit(stmt = s, setup = setup, number = 125)

0.05940659999998843

## 6. Equalizing the Grayscale Histogram

In [8]:
s = """\
equ = cv2.equalizeHist(img[0])
"""

setup = """\
import cv2
img = cv2.imread('images/lighthouse.png', 1)
"""

timeit.timeit(stmt = s, setup = setup, number = 63)

0.0032241999999769178

## 7. Converting image to grayscale

In [9]:
cv2.COLOR_BGR2GRAY
s = """\
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
"""

setup = """\
import cv2
img = cv2.imread('images/lighthouse.png', 1)
"""

timeit.timeit(stmt = s, setup = setup, number = 1270)

0.29030409999995754

## 8. Changing colorspace to HSV

In [10]:
s = """\
gray = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
"""

setup = """\
import cv2
img = cv2.imread('images/lighthouse.png', 1)
"""

timeit.timeit(stmt = s, setup = setup, number = 362)

0.604648700000098

## 9. Applying Gaussian Blur

In [13]:
s = """\
gauss = cv2.GaussianBlur(img, (3,3), cv2.BORDER_DEFAULT)
"""

setup = """\
import cv2
img = cv2.imread('images/lighthouse.png', 1)
"""

timeit.timeit(stmt = s, setup = setup, number = 75)

0.07677300000068499

## Summary:

Summarising, we get the following times:

(Values rounded off to 4 decimal points)

1. Loading images: `0.9200s`
2. Saving images: `2.0966s`
3. Resizing (Scaling) images: `0.1671s`
4. Rotating Images: `0.7836s`
5. Getting the histogram (Grayscale Histogram): `0.0594s`
6. Equalizing the Grayscale Histogram: `0.0032s`
7. Converting image to grayscale: `0.2903s`
8. Changing colorspace to HSV: `0.6046s`
9. Applying Gaussian Blur: `0.0768s`

## Table for comparison

Here is the table of the Benchmarks:

| Testcase/Framework               | Images.jl | Pillow (PIL) | OpenCV   |
|----------------------------------|-----------|--------------|----------|
| Loading images                   | 5.0217s   | 0.9200s      | 0.0796s  |
| Saving images                    | 5.0001s   | 2.0966s      | 15.6030s |
| Resizing (Scaling) images        | 5.0767s   | 0.1671s      | 0.4269s  |
| Rotating images                  | 4.9978s   | 0.7836s      | 0.6695s  |
| Calculating Greyscale Histogram  | 4.9971s   | 0.0594s      | 0.2962s  |
| Greyscale Histogram equalization | 5.0531s   | 0.0032s      | 0.3522s  |
| Converting to Greyscale          | 4.9924s   | 0.2903s      | 1.0792s  |
| Changing colorspace to HSV       | 4.9963s   | 0.6046s      | 9.1510s  |
| Applying Gaussian Blur           | 5.0297s   | 0.0768s      | 2.4995s  |

First off, we can see that BenchmarkTools.jl sets the number of samples such that the benchmarks all take 5 seconds in total. We find that Pillow (PIL) consistently gives sub zero-times for the operations with the one outlier of saving images, while OpenCV varies wildly and takes signficantly more time for some basic tasks such as opening, changing color spaces, converting to grayscale and HSV. 

Comparing Images.jl to the other 2 frameworks, Images.jl is never the fastest, which is a point where significant improvement can be made. However, Images.jl compares well with the two other framework when the operation includes changing the colorspace and saving images. Another point of concern is the fact that Images.jl is the slowest of the 3 frameworks in 7/9 of the tests. This might be due to OpenCV's powerful C and C++ bindings, or because of the powerful and fast `numpy` backend that Python has.